# CS918 - CW1 Jupyter Notebook

#### Assumptions:
* The corpus is expected to be named 'signal-news1.jsonl' and should be present in the same folder as the python script
* The nltk modules are assumed to be in the path 'modules/cs918/nltk_data/'

#### Notes:
* The script is written in python 3 and there should be no issues running it with python 3, however since I did this assignment on the lab machines in dcs, I couldn't get jupyter working with a python 3 kernel, so I could only test if this works with python2.
* If you are going to run this notebook with python3, then please remove or comment out the \_\_future\_\_ import below.
* The running time of the script/notebook for me on the dcs lab machines is about 5 minutes with the POS tagger disabled and 10 minutes with the POS tagger enabled. If you want to run the script/notebook with the POS tagger enabled, then please uncomment the lines in the tag_lemmatize method
* The perplexity of my model is about 357 without the POS tagger and 318 with the POS tagger


### Imports + Globals

In [ ]:
import json 
import re
import nltk
import math
import timeit
from __future__ import print_function 
from nltk.tag.perceptron import PerceptronTagger
from nltk.corpus import wordnet
from collections import Counter
from collections import defaultdict

nltk.data.path.append('/modules/cs918/nltk_data/')
UNK = "<UNK>"

## Part A: Text Preprocessing

The first thing to do is read all the JSON data into a list and then add each line of content separately into a list

In [ ]:
with open("signal-news1.jsonl") as f:
    json_file = f.readlines()

raw_content = []
for line in json_file:
    raw_content.append(json.loads(line)['content'].lower())


### Preprocessing the text
We then preprocess the raw content, story by story using regex.
First, all of the URLs are removed, then all of the numbers, the all non alphanumeric characters and finally, words made up of only 1 character

In [ ]:
def preprocess(content):
    urls = re.compile(r"\b(https?://)?([a-z_\-0-9]+)(\.[a-z_\-0-9]+)+(/\S*)*\b")
    content = [re.sub(urls, "", x) for x in content]
    #print ("printing content without urls \n", content[417])

    digits = re.compile(r"\b[0-9]+\b")
    content = [re.sub(digits, "", x) for x in content]
    #print ("printing content without full numbers \n", content[417])

    alpha = re.compile(r"[^a-z0-9 ]")
    content = [re.sub(alpha, "", x) for x in content]
    #print ("printing only alphachars and spaces \n", content[417])

    onechar = re.compile(r"\b\w{1}\b")
    content = [re.sub(onechar, "", x) for x in content]
    #print ("printing word with more than 1 chars \n", content[417])

    return content    
    
content = preprocess(raw_content)


### Tokenizing and Lemmatizing the content
Then all of the content is split up and tokenised into a list of lists. This is to keep content for each story seperate

In [ ]:
raw_tokens = [] # keep lists of content seperate
for l in content:
    raw_tokens.append(l.split())


The tokenised words are then lemmatized and then stored into a new list
The words can also be POS-tagged before being lemmatized by just uncommenting out the lines in the tag_lemmatize method and commenting out the last line. Since the POS tags generate by the PerceptronTagger are treebank tags, they first need to be converted to the more simple wordnet tags before they can be used by the WordNetLemmatizer. This is done in the get_wordnet_tag function.
However, I have disabled this since it adds about 5 extra minutes to the running time.

In [ ]:
def get_wordnet_tag(treebank):
    if treebank.startswith("J"):
        return wordnet.ADJ
    elif treebank.startswith("V"):
        return wordnet.VERB
    elif treebank.startswith("N"):
        return wordnet.NOUN
    elif treebank.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def tag_lemmatize(wnl, tagger, raw_tokens):
    wordnet_tags = []
    #for word in raw_tokens:
    #    word, treebank = tagger.tag([word])[0]
    #    wordnet_tags.append((word, get_wordnet_tag(treebank)))
    #return [wnl.lemmatize(word, tag) for word, tag in wordnet_tags]
    return [wnl.lemmatize(word) for word in raw_tokens]

wnl = nltk.WordNetLemmatizer()
tagger = PerceptronTagger()
tokens = []
for l in raw_tokens:
    tokenized_story = tag_lemmatize(wnl, tagger, l)
    tokens.append(tokenized_story)

## Part B: N-Grams

### Computing the vocabulary 
We then compute the vocabulary of the corpus, once again we keep it separate for each story.
This is done using dictionaries, where each key is a unique token and its value is the number of times it appears in the  the specific story.
From here it is trivial to compute the number of tokens and the vocabulary size. 
For the number of tokens, we sum the counts for each story to get a list of the number of tokens in each story. From there the number of tokens in the whole corpus is sum the list generated.
Similary for the vocabulary size, we just get the list of keys for each story, and add them to a set. We can then find the length of that set in order to compute the size of the vocabulary size of the whole corpus.

In [ ]:
vocab = []
for story in tokens:
    story_vocab = Counter()
    for t in story:
        story_vocab[t] += 1
    vocab.append(story_vocab)

#Number of tokens N
print ("Number of tokens: ", sum([sum(v.values()) for v in vocab]))
#Vocabulary size
print ("Size of vocab: ", len({word for v in vocab for word in v.keys()}))


### Computing Trigrams
To compute the top 25 trigrams, we first need to compute all of the trigrams and their respective counts. This is done in a similar way to how we generate the vocabulary.
For each tokenized story, we generate all the trigrams for that story using the nltk.trigram function. We then add each of these trigrams to a dictionary which and update their count's if they reoccur. This is done for each story in the corpus so we end up with a dictionary that contains the counts for each trigram present in the corpus.
From here we can easily find the top 25 trigrams by simply sorting the dictionary by its values, and then taking the top 25 from that list.

#### Note: We have also generated the trigram and bigram counts per story aswell, these will be used when generating the language models later.

In [ ]:
#Top 25 trigrams
all_trigrams = Counter()
story_trigrams = []
story_bigrams = []
for story in tokens:
    tlist = nltk.trigrams(story)
    trigrams = Counter()
    for trigram in tlist:
        trigrams[trigram] += 1
    all_trigrams.update(trigrams)
    story_trigrams.append(trigrams)

    blist = nltk.bigrams(story)
    bigrams = Counter()
    for bigram in blist:
        bigrams[bigram] += 1
    story_bigrams.append(bigrams)

print("Top 25 trigrams :", sorted(list(all_trigrams.items()), key=lambda x: x[1], reverse=True)[:25])


### Computing positive and negative counts
Before we can compute the positive and negative counts in the corpus, we must first read in the positive and negative words into seperate lists and preprocess them similary to how the content is proprocessed.

In [ ]:
#Positive and negative counts
with open("positive-words.txt", "r") as pos:
    positives = set()
    for word in pos:
        positives.add(re.sub(r"[^a-z0-9 ]", "", word.rstrip("\n")))

with open("negative-words.txt", "r") as neg:
    negatives = set()
    for word in neg:
        negatives.add(re.sub(r"[^a-z0-9 ]", "", word.rstrip("\n")))


Since we have already computed the vocabulary for each story, we can simply iterate through word in a particular story, check whether it is positive or negative (or neither) and then add it's count the number of positive or negative words per story.
Once we have the positive and negative counts per story, we can then decide whether a story is positive or negative and increase the count of either negative or positive stories and then finally add the local positive and negative counts to the global ones so we can compute the total negative and positive counts of the corpus.

In [ ]:
pos_stories = 0
neg_stories = 0
pos_count = 0
neg_count = 0

def count_pos_neg(positives, negatives, vocab): 
    pos_count = 0
    neg_count = 0
    for token, count in vocab.items():
        if token in positives:
            pos_count += count
        if token in negatives:
            neg_count += count
    return pos_count, neg_count

for story in vocab:
    local_pos, local_neg = count_pos_neg(positives, negatives, story)
    if local_pos > local_neg:
        pos_stories += 1
    if local_pos < local_neg:
        neg_stories += 1
    pos_count += local_pos
    neg_count += local_neg
    
print("Number of positive words: {}, Number of negative words: {}".format(pos_count, neg_count))
print("Number of positive stories: {}, Number of negative stories: {}".format(pos_stories, neg_stories))


## Part C: Language Models
### Creating the models
Since we are using "stupid backoff" for our trigram language model we create 3 models, a trigram model, a bigram model and a unigram model from the 16000 items in the corpus. These are constructed using the per story trigram and bigram counts that we computed earlier
Also in the unigram model, we replace all tokens with a count of 1 or less to the "UNK" token. 

In [ ]:
#Part C: Language Models
tri_model = defaultdict(lambda: defaultdict(lambda: 0))
bi_model = defaultdict(lambda: defaultdict(lambda: 0))
uni_model = defaultdict(lambda: 0)

for i in range(0, 16000):
    for (x, y, z), count in story_trigrams[i].items():
        tri_model[(x, y)][z] += count
    for (x, y), count in story_bigrams[i].items():
        bi_model[x][y] += count
    for t, count in vocab[i].items():
        if count <= 1:
            uni_model[UNK] += count
        else:
            uni_model[t] += count


### Forming a sentence
Once we have computed our language models, we can form a sentence with the bigram ("is","this") by using our trigram model to get all the possible trigrams in our model that start with the bigram, then sorting the list produced to find the most likely one. We then append the most likely word to our sentence and then change our bigram to last two words in the sentence and continue the same process untill we get a sentence with 10 words.

In [ ]:
def form_sentence(tri_model, bigram, length):
    i = 0
    sentence = [bigram[0], bigram[1]]
    while len(sentence) < length:
        all_trigrams = tri_model[(sentence[i], sentence[i+1])]
        most_likely = sorted(all_trigrams.items(), key=lambda x: x[1], reverse=True)[0]
        sentence.append(most_likely[0])
        i += 1
    return sentence

bigram = ("is","this")
bigram = tag_lemmatize(wnl, tagger, bigram)
print (*form_sentence(tri_model, bigram, 10), sep=" ")

### Computing Perplexity
To compute perplexity we first need to be able to compute the probability of a trigram in our model. We do this using "stupid" backoff, which is as follows:
If the trigram (w1, w2, w3) exists in the in model, then return the MLE(Maximum Likelihood Estimate) for that trigram.
If not, we check if the bigram (w2, w3) exists, then return the MLE of that trigram, but but multiplied by the alpha value of 0.4
If the bigram does not exist either, we then, return MLE for the unigram (w3) instead, but this time the alpha is 0.4 \* 0.4.
If still, the unigram does not exist in the model either, we then return the MLE of the UNK token, again multiplier by the alpha of 0.4 \* 0.4

In [ ]:
def stupid_backoff(tri_model, bi_model, uni_model, trigram):
    stupid = 0.4
    x, y, z = trigram
    if tri_model[(x, y)][z] > 0:
        return tri_model[(x, y)][z]/bi_model[x][y]
    elif bi_model[y][z] > 0:
        return stupid * (bi_model[y][z]/sum(bi_model[y].values()))
    elif uni_model[z] > 0:
        return stupid*stupid*(uni_model[z]/sum(uni_model.values()))
    else:
        return stupid*stupid*(uni_model[UNK]/sum(uni_model.values()))


Once we can compute the probability of a trigram, we can now evaluate the perplexity of the model against the remaing rows of the corpus.
The perplexity is calculated as 2 to power of minus 1 over the sum of the log probabilities for each trigram in the test data.

In [ ]:
def calc_perplexity(tri_model, bi_model, uni_model, test):
    perplexity = 0.0
    num_trigrams = 0
    unseen = 0
    test_trigrams = Counter()
    for row in test:
        test_trigrams.update(row)

    for trigram, count in test_trigrams.items():
        prob = stupid_backoff(tri_model, bi_model, uni_model, trigram)
        if prob != 0:
            perplexity += count*math.log(prob, 2)
        num_trigrams += count
    return math.pow(2, -(perplexity/num_trigrams))

print("perplexity of model is: ", calc_perplexity(tri_model, bi_model, uni_model, story_trigrams[16000:]))